In [ ]:
#Nirmal Minz : lci2020076
import numpy as np
import math
import random

def calculate_flow_interference(task_index, sensor_index, assigned_nodes, sensor_data, communication_data, deadline_data, flow_data, max_time_slot):
    communication_delay = 0

    for neighbor_index in range(len(sensor_data)):
        if neighbor_index != sensor_index and deadline_data[neighbor_index] >= max_time_slot and sensor_data[assigned_nodes[task_index]].count(neighbor_index) > 0:
            flow_interference = 0
            for flow_index in range(len(flow_data)):
                if flow_data[flow_index][0] == task_index and flow_data[flow_index][1] == neighbor_index:
                    flow_interference += flow_data[flow_index][2]
            if flow_interference + communication_data[sensor_index][neighbor_index] ** 2 != 0:
                sinr = sensor_data[assigned_nodes[task_index]][sensor_index] / (flow_interference + communication_data[sensor_index][neighbor_index] ** 2)
            else:
                sinr = 0  # Or any other appropriate value based on your requirements

            # sinr = sensor_data[assigned_nodes[task_index]][sensor_index] / (flow_interference + communication_data[sensor_index][neighbor_index] ** 2)
            communication_delay += sinr

    return communication_delay


In [ ]:

#Pallavi Chauhan : lci2020073
def calculate_fitness(particle, task_data, sensor_data, deadline_data, communication_data, reliability_data, flow_data):
    assigned_nodes = [-1] * len(task_data)

    for task_index, sensor_index1 in enumerate(particle):
        sensor_index = int(sensor_index1)
        # print(particle, sensor_index)
        while assigned_nodes[sensor_index] >= 0:
            min_distance = math.inf
            min_sensor_index = -1
            for neighbor_index, neighbor_reliability in enumerate(sensor_data[sensor_index]):
                if assigned_nodes[neighbor_index] < 0:
                    distance = (
                        neighbor_reliability / (communication_data[sensor_index][neighbor_index] ** 2)
                    ) if communication_data[sensor_index][neighbor_index] != 0 else float('inf')
                    if distance < min_distance:
                        min_distance = distance
                        min_sensor_index = neighbor_index
            sensor_index = min_sensor_index

        particle[task_index] = sensor_index
        assigned_nodes[sensor_index] = task_index

    total_fitness = 0
    for sensor_index in range(len(sensor_data)):
        tasks_on_node = [task_index for task_index, node_index in enumerate(assigned_nodes) if node_index == sensor_index]

        execution_time = 0
        communication_delay = 0

        for task_index in tasks_on_node:
            task_fragments = task_data[task_index]
            task_time_slots = [slot for fragment in task_fragments for slot in fragment]
            max_time_slot = max(task_time_slots)
            execution_time += max_time_slot

            communication_delay += calculate_flow_interference(task_index, sensor_index, assigned_nodes, sensor_data, communication_data, deadline_data, flow_data, max_time_slot)
            
        node_reliability = max(sensor_data[sensor_index])
        node_fitness = node_reliability / (execution_time + communication_delay + deadline_data[sensor_index])
        total_fitness += node_fitness

    return total_fitness




In [ ]:

# Devaryan LCI2020072
import random
import numpy as np

def initialize_particles(num_particles, max_task_num, max_sensor_num):
    particles = []
    for _ in range(num_particles):
        particle = []
        for j in range(max_task_num):
            column_values = random.sample(range(max_sensor_num), max_task_num)
            particle.append(column_values[j])
        particles.append(particle)
    return np.array(particles)

def initialize_velocities(num_particles, max_task_num, max_sensor_num):
    return np.zeros((num_particles, max_task_num), dtype=int)


def update_velocities(velocities, particles, best_particles, global_best_particle, inertia_weight, cognitive_weight, social_weight, velocity_max):
    cognitive_component = cognitive_weight * np.random.uniform(0, 1, size=particles.shape) * (best_particles - particles)
    social_component = social_weight * np.random.uniform(0, 1, size=particles.shape) * (global_best_particle - particles)
    new_velocities = np.clip(velocities * inertia_weight + cognitive_component + social_component, -velocity_max, velocity_max)
    return new_velocities


def update_particles(particles, velocities, max_sensor_num):
    new_particles = np.clip(particles + velocities, 0, max_sensor_num - 1)
    return new_particles


def update_best_particles(particles, particle_fitness, best_particles, best_particle_fitness):
    improved_indices = particle_fitness < best_particle_fitness
    best_particles[improved_indices] = particles[improved_indices]
    best_particle_fitness[improved_indices] = particle_fitness[improved_indices]
    return best_particles, best_particle_fitness


def update_global_best_particle(best_particles, best_particle_fitness):
    global_best_particle_index = np.argmin(best_particle_fitness)
    global_best_particle = best_particles[global_best_particle_index]
    return global_best_particle_index, global_best_particle


def adaptive_discrete_particle_swarm_optimization(num_iterations, num_particles, max_task_num, max_sensor_num, task_data, sensor_data, deadline_data, communication_data, reliability_data, flow_data):
    inertia_weight_min = 0.4
    inertia_weight_max = 0.9
    cognitive_weight_min = 1.5
    cognitive_weight_max = 2.5
    social_weight_min = 1.5
    social_weight_max = 2.5
    velocity_max = max_sensor_num - 1

    particles = initialize_particles(num_particles, max_task_num, max_sensor_num)
    particle_fitness = np.array([calculate_fitness(p, task_data, sensor_data, deadline_data, communication_data, reliability_data, flow_data) for p in particles])

    best_particles = np.copy(particles)
    best_particle_fitness = np.copy(particle_fitness)
    global_best_particle_index, global_best_particle = update_global_best_particle(best_particles, best_particle_fitness)

    velocities = initialize_velocities(num_particles, max_task_num, max_sensor_num)

    prev_global_best_particle = None  # Track previous global best particle

    for iteration in range(num_iterations):
        inertia_weight = inertia_weight_max - (inertia_weight_max - inertia_weight_min) * (iteration / num_iterations)
        cognitive_weight = cognitive_weight_max - (cognitive_weight_max - cognitive_weight_min) * (iteration / num_iterations)
        social_weight = social_weight_max - (social_weight_max - social_weight_min) * (iteration / num_iterations)

        velocities = update_velocities(velocities, particles, best_particles, global_best_particle, inertia_weight, cognitive_weight, social_weight, velocity_max)

        particles = update_particles(particles, velocities, max_sensor_num)

        particle_fitness = np.array([calculate_fitness(p, task_data, sensor_data, deadline_data, communication_data, reliability_data, flow_data) for p in particles])

        best_particles, best_particle_fitness = update_best_particles(particles, particle_fitness, best_particles, best_particle_fitness)

        global_best_particle_index, global_best_particle = update_global_best_particle(best_particles, best_particle_fitness)

        if prev_global_best_particle is not None and np.array_equal(global_best_particle, prev_global_best_particle):
            # Stop the loop if the current best particle is the same as the previous best particle
            break

        prev_global_best_particle = np.copy(global_best_particle)

    return best_particles[global_best_particle_index], best_particle_fitness[global_best_particle_index]

# Example usage
num_iterations = 400
num_particles = 40
max_task_num = 10
max_sensor_num = 10
for i in range(20):
# Generate random task data

  max_fragments = 4
  max_time_slots = 5

  # Generate random task data with fragments and time slots
  task_data = []
  timeslots = []
  for i in range(max_task_num):
      fragments = []
      for j in range(max_fragments):
          time_slots = [random.uniform(1, 10) for k in range(max_time_slots)]
          fragments.append(time_slots)
          timeslots.append(time_slots)
      task_data.append(fragments)
  print(task_data)
  sensor_data = [[random.uniform(0, 10) for j in range(max_sensor_num)] for i in range(max_sensor_num)]

  deadline_data = [random.randrange(0, 1) for i in range(max_task_num)]


  # Generate random communication data
  communication_data = [[random.randrange(1, 2) for j in range(max_sensor_num)] for i in range(max_sensor_num)]
  print(communication_data)
  # print('\n')

  # Generate random reliability data
  reliability_data = [[random.uniform(0, 1) for j in range(max_sensor_num)]for i in range(max_sensor_num)]
  print(reliability_data)
  # print('\n')

  reliable_sensors = [i for i in range(len(reliability_data)) if all(r >= 0.0 for r in reliability_data[i])]
  print(reliable_sensors)
  # print('\n')
  sensor_data_filtered = [[sensor_data[i][j] for j in reliable_sensors] for i in reliable_sensors]
  print(sensor_data_filtered)
  # print('\n')
  deadline_data_filtered = [deadline_data[i] for i in reliable_sensors]
  print(deadline_data_filtered)
  # print('\n')
  communication_data_filtered = [[communication_data[i][j] for j in reliable_sensors] for i in reliable_sensors]
  print(communication_data_filtered)
  # print('\n')
  reliability_data_filtered = [reliability_data[i] for i in reliable_sensors]
  print(reliability_data_filtered)
  # print('\n')

  flow_data = [(random.randrange(max_task_num), random.choice(reliable_sensors), random.uniform(0, 10)) for _ in range(max_task_num)] 
  print('\n')

  result = adaptive_discrete_particle_swarm_optimization(num_iterations, num_particles, max_task_num, max_sensor_num, task_data, sensor_data, deadline_data, communication_data, reliability_data, flow_data)
  print(result)